# Entendimiento de negocio (CRISP‑DM)

**Proyecto:** Optimización del transporte en Nueva York (Green Taxi)

## 1) Contexto y problema
La Comisión de Taxis y Limusinas de Nueva York (TLC) mantiene registros masivos de viajes. Queremos usar los registros de Green Taxi para entender y predecir atributos clave del viaje — por ejemplo: duración estimada y tarifa total — con el fin de mejorar la experiencia del pasajero, optimizar rutas y apoyar la planificación urbana.

## 2) Objetivos de negocio
- Objetivo primario (negocio): Predecir la duración y/o tarifa de un viaje con precisión suficiente para informar a pasajeros y optimizar la asignación de vehículos.
- Objetivos secundarios: detectar anomalías (viajes atípicos), estimar demanda por zonas/horas y evaluar efectos de tarifas adicionales (congestion_surcharge, cbd_congestion_fee).

## 3) Stakeholders
- TLC (regulador): políticas y planificación.
- Operadores (empresas de dispatch): asignación y flotas.
- Conductores: optimización de rutas y tiempos activos.
- Pasajeros: estimaciones de coste/tiempo al reservar.
- Equipo de datos/ML: entrega de modelos y monitoreo.

## 4) Criterios de éxito / métricas
- Métrica principal (regresión): MAE y RMSE para duración y tarifa.
- Objetivo orientativo: MAE de tiempo ≤ X minutos (definir tras EDA) y MAE de tarifa ≤ Y dólares.
- Métricas operativas: % de predicciones dentro de ±10% del valor real, cobertura por zona/hora, latencia de inferencia < Z ms para uso en tiempo real.

## 5) Preguntas clave
- ¿Qué variables explican mejor la duración/tarifa? (hora, distancia, zona pickup/dropoff, passenger_count, ratecodeID)
- ¿Cómo afectan los recargos (congestion_surcharge, cbd_congestion_fee) la tarifa total?
- ¿Hay patrones estacionales o por hora que afecten precisión?

## 6) Fuentes de datos y disponibilidad
- Archivo principal (Green Taxi parquet):

  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-06.parquet
- Variables relevantes: lpep_pickup_datetime, lpep_dropoff_datetime, trip_distance, fare_amount, tip_amount, tolls_amount, congestion_surcharge, cbd_congestion_fee, passenger_count, payment_type, ratecodeID, PULocationID, DOLocationID.

## 7) Supuestos y limitaciones
- Supuesto: timestamps en zona horaria consistente (validar y convertir si es necesario).
- Limitación: datos perdidos/erróneos (distancias cero, tarifas negativas) requerirán limpieza y reglas de filtrado.
- Los datos históricos reflejan comportamiento normal; eventos extraordinarios (clima extremo, huelgas) pueden degradar desempeño.

## 8) Riesgos
- Calidad de datos y cambios en el esquema (nuevos cargos introducidos en 2025).
- Sesgo geográfico o por horario que reduzca generalización.

## 9) Plan inicial (próximos pasos)
1. EDA: volumen, nulos, distribuciones, correlaciones y limpieza básica (filtrar viajes inválidos).
2. Ingeniería de features: duración real, velocidad media, extracción de variables temporales (hora, día, is_weekend), agregados por zona/hora.
3. Modelado: baseline (regresión lineal), modelos tree‑based (Random Forest, XGBoost) y evaluación con CV temporal.
4. Validación: MAE/RMSE por zona y por hora, pruebas de estabilidad temporal.
5. Despliegue piloto y monitoreo (drift, calidad).

---

*Nota:* A continuación hay una celda de código que carga el dataset desde la URL pública. Recomendado: ejecutar primero la celda de carga y luego las de EDA.


In [3]:
import os
import pandas as pd

# URL del dataset
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-06.parquet"

# Cargar el archivo parquet desde la URL
data = pd.read_parquet(url, engine="pyarrow")

In [ ]:
data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-06-01 00:33:43,2025-06-01 01:04:33,N,2.0,74,132,1.0,19.60,70.0,...,0.5,19.61,6.94,NaN,1.0,98.05,1.0,1.0,0.0,0.0
1,2,2025-06-01 00:07:45,2025-06-01 00:14:52,N,1.0,75,74,2.0,1.37,9.3,...,0.5,0.00,0.00,NaN,1.0,11.80,2.0,1.0,0.0,0.0
2,2,2025-06-01 00:24:07,2025-06-01 00:48:24,N,1.0,83,83,1.0,4.11,25.4,...,0.5,0.00,0.00,NaN,1.0,27.90,2.0,1.0,0.0,0.0
3,2,2025-06-01 00:00:14,2025-06-01 00:08:29,N,1.0,97,49,1.0,1.29,9.3,...,0.5,2.36,0.00,NaN,1.0,14.16,1.0,1.0,0.0,0.0
4,2,2025-06-01 00:31:15,2025-06-01 00:43:35,N,1.0,66,25,1.0,1.97,13.5,...,0.5,0.00,0.00,NaN,1.0,16.00,1.0,1.0,0.0,0.0
